# Image noise reduction

## Problem Overview

We use an annealing machine to restore the original image from an image with added noise.

We attempt to reduce noise based on the following assumptions:

*   The original image and the image with noise have relatively large overlaps.
*   In the original image, neighboring pixels often have the same color.

We use black and white images here as a simple example. 
Since the data of a pixel can be represented by a binary value of black and white, the value of each pixel can be expressed a binary variable.
By formulating an objective function which represents the interaction between pixels expressing the above assumptions, the original image can be estimated by finding the optimal value of this function.

## Constructing the Objective Function

We let $V$ be the set of pixels, and let $i\in V$ be the index representing each pixel.
We then let the binary valued data representing the input pixels with noise be $y$, where the value of each pixel is expressed as follows:

$$
y_{i} = \left\{
\begin{align}
&+1 \quad\text{(white)}\\
&-1 \quad \text{(black)}
\end{align}
\right. \quad
i\in V\\
$$

Also, the binary Ising variables corresponding to the output pixels are represented as follows:

$$
s_{i} = \left\{
\begin{align}
&+1 \quad\text{(White)}\\
&-1 \quad \text{(Black)}
\end{align}
\right. \quad
i\in V\\
$$

Based on the assumption that the input and output images are are relatively close (i.e., there is not much noise), we need the effect that the input and output pixels have the same values. In other words, we introduce an objective function such that $s_i$ and $y_i$ become smaller when they have the same value. For example, it is given as follows:

$$
f_1 = - \sum_{i\in V} y_{i} s_{i}
$$

Since the value of the above objective function decreases when $y_{i}$ and $s_{i}$ have the same value and increases when they have different values, $f_1$ takes the minimum value when $y_{i} = s_{i}$ for all $i\in V$. However, since the input image has noise on it, the noise cannot be reduced if the output image is the same as the input image.

We thus consider the assumption that neighboring pixels tend to be the same color.
In other words, we introduce an objective function that reduces the value when neighboring output pixels have the same value. For example, it is given as follows:
 
$$
f_2 = -\sum_{(i,j)\in E} s_i s_j
$$

Here, the set of adjacent pixel pairs is defined as $E$. If all the output pixels have the same value, $f_2$ takes the smallest value. However, if all pixels have the same value, every pixel will turn out to be white or black, and the information in the original image will be lost.

Therefore, by appropriately adding $f_1$ and $f_2$ together, we try to remove the pixels that are considered to be noise while making the output image close to the input image.

$$
\begin{align}
f & = f_1 + \eta f_2\\
&=- \sum_{i\in V}y_is_i - \eta \sum_{(i,j)\in E}s_i s_j
\end{align}
$$

Here, we have introduced the parameter $\eta>0$. This allows us to adjust the relative strength of $f_1$ and $f_2$. The larger the value of $\eta$ is, the stronger the effect of the noise reduction term is.

By minimizing this objective function and interpreting the values of the Ising variables $s$ as the values of the pixels, we can obtain an image with reduced noise.


## Reference
* [Annealing Cloud Web: Demo App](https://annealing-cloud.com/ja/play/demoapp/noise.html)
* [Annealing Cloud Web: Explanation of Image Noise Reduction](https://annealing-cloud.com/ja/tutorial/2.html)

## Loading an Image

First, we define a function to download an image data and a function to convert the downloaded image to an array of Ising variables.

In [ ]:
from PIL import Image
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

# Create an Ising array of the original image
img = Image.open("sample.png")
x = np.where(np.array(img) >= 128, 1, -1)
plt.imshow(x, cmap="gray")
plt.show()
print(x.shape)
print(x)

## Creating an Image with Noise

Next, we define a function that represents noise by randomly selecting pixels and inverting their values.

In [ ]:
# Function to add noise to an image
def get_noisy_img_array(img_array: np.ndarray, noise_ratio: float = 0.02) -> np.ndarray:
    rng = np.random.default_rng()
    # Select a pixel to add noise
    invert_pixels = rng.choice(
        [1, -1], size=img_array.shape, p=[1 - noise_ratio, noise_ratio]
    )

    return img_array * invert_pixels

In [ ]:
# Create an Ising array of image with noise
y = get_noisy_img_array(x)
plt.imshow(y, cmap="gray")
plt.show()

## Creating the Ising Variable Array

Next, we create an array `s` of Ising variables. If the input image data `y` is a $h\times w$ two-dimensional array, the Ising variable `s` corresponding to the output image is also a $h\times w$ two-dimensional array.

We use `gen_symbols` to generate the variables. We specify the type of the variable as `IsingPoly` here because the polynomial of the Ising variables will be the objective function in the end. Since `gen_symbols` can create an array of variables in the same form as the array of the input image data `y`, we use this feature as follows.

In [ ]:
from amplify import VariableGenerator

gen = VariableGenerator()
# Create an Ising variable in the shape of original image
s = gen.array("Ising", shape=y.shape)

## Objective Function

We construct the objective function using the array of the input image data $y$ and the Ising variable array $s$ corresponding to the output image.


In [ ]:
# Strength parameter
eta = 0.333

# Calculate the objective function f

# Image after noise removal often matches image before removal
# -> - we want to reduce [\sum_{i\in V} y_{i} s_{i}]
f1 = -(s * y).sum()

# Neighboring pixels are often the same color in the image after noise removal
# -> - we want to make [\sum_{(i,j)\in E} s_i s_j] smaller
f2 = -((s[:, :-1] * s[:, 1:]).sum() + (s[:-1, :] * s[1:, :]).sum())

# Set the objective function so that the values of the above two functions become small at the same time
f = f1 + eta * f2

## Setting Up the Client and Running the Machine

Next, we set up the client and search for the solution corresponding to the minimum value of the objective function, by an Ising machine.

In [ ]:
from amplify import FixstarsClient, solve
from datetime import timedelta

# Setting Fixstars Amplify AE as a client
client = FixstarsClient()
client.parameters.timeout = timedelta(milliseconds=2000)  # Timeout is 2 second
# client.token = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"  # If you use it in a local environment, please enter the access token for Amplify AE

# Solve the problem
result = solve(f, client)

## Obtaining the Solutions and Displaying the Results

Finally, we substitute the found solution to the Ising variable $s$ to obtain the data of the output image.

Comparing with the input image, we can see that the noise has been reduced.

In [ ]:
# Assign the solution to the Ising variable array
output = s.evaluate(result.best.values)

plt.imshow(output, cmap="gray")  # Restored image
plt.show()

plt.imshow(x, cmap="gray")  # Original image
plt.show()

plt.imshow(y, cmap="gray")  # Image with noise
plt.show()